In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

In [5]:
y = pd.read_csv(r"C:\Users\anant\Documents\GitHub\Taxonomic_classifier\Data\Taxonomic_data.csv", usecols=['Species'])
x = pd.read_csv(r"C:\Users\anant\Documents\GitHub\Taxonomic_classifier\Data\k(4)mer_data.csv").drop("Unnamed: 0", axis=1)

In [6]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
xx = pca.fit_transform(x)
pca.inverse_transform(xx)
pca.explained_variance_ratio_.sum()
print(f"{x.shape[1]} columns originally present, \n{pca.n_components_} account for 95 percent variance ")
print(f"corresponds to {100*pca.n_components_/x.shape[1]}% of total")

256 columns originally present, 
115 account for 95 percent variance 
corresponds to 44.921875% of total


In [7]:
train_x, test_x, train_y, test_y = train_test_split(xx, y, test_size=0.2, random_state=7)

In [8]:
train_y.describe()

,Species
count,10601
unique,2351
top,Escherichia coli
freq,712


In [9]:
train_x.shape

(10601, 115)

In [10]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(y)
test_y = lb.transform(test_y)
train_y = lb.transform(train_y)


In [11]:
model = Sequential()
model.add(Dense(20, input_shape =(None, train_x.shape[1]) , activation="relu"))
model.add(Dense(len(y.Species.unique()),activation='softmax'))

In [12]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, None, 20)          2320      
                                                                 
 dense_1 (Dense)             (None, None, 2621)        55041     
                                                                 
Total params: 57,361
Trainable params: 57,361
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(train_x, train_y, epochs = 30)

Epoch 1/30
332/332 [==============================] - 1s 3ms/step - loss: 5.0120 - accuracy: 0.2898
Epoch 2/30
332/332 [==============================] - 1s 3ms/step - loss: 2.4786 - accuracy: 0.5464
Epoch 3/30
332/332 [==============================] - 1s 3ms/step - loss: 1.5555 - accuracy: 0.6699
Epoch 4/30
332/332 [==============================] - 1s 3ms/step - loss: 1.0453 - accuracy: 0.7567
Epoch 5/30
332/332 [==============================] - 1s 3ms/step - loss: 0.7557 - accuracy: 0.8061
Epoch 6/30
332/332 [==============================] - 1s 3ms/step - loss: 0.5902 - accuracy: 0.8400
Epoch 7/30
332/332 [==============================] - 1s 2ms/step - loss: 0.4854 - accuracy: 0.8607
Epoch 8/30
332/332 [==============================] - 1s 2ms/step - loss: 0.4206 - accuracy: 0.8744
Epoch 9/30
332/332 [==============================] - 1s 3ms/step - loss: 0.3741 - accuracy: 0.8851
Epoch 10/30
332/332 [==============================] - 1s 3ms/step - loss: 0.3364 - accuracy: 0.8938

In [14]:
model.evaluate(test_x, test_y)

83/83 [==============================] - 0s 2ms/step - loss: 2.9396 - accuracy: 0.8276


[2.939554452896118, 0.8276122212409973]

In [15]:
from sklearn import metrics
f1 = metrics.f1_score(lb.inverse_transform(test_y), lb.inverse_transform(model.predict(test_x)), average = 'weighted')
f1

0.8103175044567315

In [16]:
i = 0
mismatch = 0
npredictions = 0
predictions = []
mismatch_species = []
mismatch_species_pred = []
y_tested = []
for y in test_y:
    a = model.predict(test_x[i:i+1]) 
    if a.max()>0.8:
        sp = lb.inverse_transform(np.array([y]))
        p_sp = lb.inverse_transform(a)
        if p_sp!=sp :
            mismatch_species.append(sp)
            mismatch_species_pred.append(p_sp)
            mismatch += 1
        npredictions += 1
        predictions.append(p_sp)
        y_tested.append(sp)
    i  += 1
print(mismatch)

158


In [17]:
from sklearn import metrics
f1 = metrics.f1_score( lb.inverse_transform(test_y), lb.inverse_transform(model.predict(test_x)), average='weighted')
f1

0.8103175044567315

In [18]:
lb.inverse_transform(test_y)

array(['Lactobacillus reuteri', 'Staphylococcus aureus',
       'Moraxella catarrhalis', ..., 'Mycobacterium vanbaalenii',
       'Colwellia psychrerythraea', 'Helicobacter cinaedi'], dtype='<U50')

In [19]:
i, npredictions, mismatch

(2651, 2144, 158)

In [20]:
i
# 462

2651

In [22]:
model.save("k4-net")

INFO:tensorflow:Assets written to: k4-net\assets
